Import necessary modules.

In [1]:
import random
import math
import simpy

Calculate N. Last three digits of our student numbers are 210, 024 and 123.

In [2]:
def get_N():
    s = 210 + 24 + 123
    if s > 1000:
        return s
    elif s > 10:
        return s + 1000
    else:
        return s * 300

K, Lambda, Mu1 and Mu2 are already given.

In [3]:
N = get_N()

K = math.ceil(N/12)

Lambda = N / 300

Mu1 = 1/6

Mu2 = 1/10

Define r as given in project description and calculate Mu3.
### Change `RANDOM_SEED` to change seed.

In [4]:
RANDOM_SEED = 1234
random.seed(RANDOM_SEED)

r = random.uniform(1, 2)

Mu3 = 1 / (1/Mu1 * r)

Define the variables that we are going to need.

In [5]:
L = []          # L[t] = number of patients in the system at time t
L_hosp = []     # L_hosp[t] = number of patients in the hospital at time t
L_home = []     # L_home[t] = number of patients in their home at time t
A = [0, 0]      # A[n] = interarrival time between patients n-1 and n
S = {}          # S[n] = healing time (= total time spent in system) of patient n
Arrivals = {}   # Arrivals[n] = arrival time of patient n
Leaves = {}     # Leaves[n] = Leaving time of patient n
Log = []        # Used to log all events

Define the log function. This function keeps track of events.

In [6]:
def log(clock, event_type, patient_type, patient_index):
    Log.append({"clock": clock, "event_type": event_type, "patient_type": patient_type, "patient_index": patient_index})


 ### The class definition for the patients arriving at the modeled system. 
 - Constructor: `__init__` function: When they are created, they immediatelly initiate a call (i.e. activate the call process).
 - `get_sick2` function:  This function corresponds to initial patients at the hospital, so all patients directly goes to the hospital.
 - `get_sick` function: 
    - A patient stays at home with 80% probability. His/her healing time is exponentially distributed with Mu2.
    - A patient goes to hospital with 20% probability.
       - If s/he is accepted to hospital, his/her healing time is exponentially distributed with Mu1.
       - If s/he is not accepted to hospital due to full capacity, his/her healing time is exponentially distributed with Mu3.

In [7]:
class Patient:
    def __init__(self, index,isInitial):
        self.index = index
        self.name = f"Patient {index}"
        if isInitial:
            self.action = env.process(self.get_sick2())
        else:
            self.action = env.process(self.get_sick())

    def get_sick2(self):
        Arrivals[self.index] = env.now
        with hospital.request() as req:
            results = yield req | env.timeout(0)
            if req in results:
#                 log(env.now, "Arrive", 1, self.index)
                healing_time = random.expovariate(Mu1)
                yield env.timeout(healing_time)
                S[self.index] = healing_time
                Leaves[self.index] = env.now
                log(env.now, "Leave", 1, self.index)
                
    def get_sick(self):
        Arrivals[self.index] = env.now
        rand = random.random()
        if rand < 0.8:
            # stay Home
            with homes.request() as req:
                yield req
                log(env.now, "Arrive", 2, self.index)
                healing_time = random.expovariate(Mu2)
                yield env.timeout(healing_time)
                S[self.index] = healing_time
                Leaves[self.index] = env.now
                log(env.now, "Leave", 2, self.index)
        else:
            #go hospital
            with hospital.request() as req:
                # accepted to hospital
                results = yield req | env.timeout(0)
                if req in results:
                    log(env.now, "Arrive", 1, self.index)
                    healing_time = random.expovariate(Mu1)
                    yield env.timeout(healing_time)
                    S[self.index] = healing_time
                    Leaves[self.index] = env.now
                    log(env.now, "Leave", 1, self.index)
                else:
                    # rejected due to full capacity
                    with homes.request() as req2:
                        yield req2
                        log(env.now, "Arrive", 3, self.index)
                        healing_time = random.expovariate(Mu3)
                        yield env.timeout(healing_time)
                        S[self.index] = healing_time
                        Leaves[self.index] = env.now
                        log(env.now, "Leave", 3, self.index)

        if self.index == N:
            L_monitor_process.interrupt()
            L_hosp_monitor_process.interrupt()
            L_home_monitor_process.interrupt()


`N` patients are generated with exponential distribution. Starting index determines the number of patients in hospital at the beginning.

In [8]:
def patient_generator(starting_index=1,patient_count=N ):
    for i in range(1,starting_index):
        Patient(i,True)
    for i in range(starting_index, patient_count+starting_index):
        Patient(i,False)
        inter_arrival = random.expovariate(Lambda)
        yield env.timeout(inter_arrival)
        A.append(inter_arrival)


This function keeps track of the number of patients in the system every second.

In [9]:
def L_monitor():
    try:
        while True:
            L.append(hospital.count + homes.count)
            yield env.timeout(1)
    except simpy.Interrupt:
        pass

This function keeps track of the number of patients in the hospital every second.

In [10]:
def L_hosp_monitor():
    try:
        while True:
            L_hosp.append(hospital.count)
            yield env.timeout(1)
    except simpy.Interrupt:
        pass

This function keeps track of the number of patients in the homes every second.

In [11]:
def L_home_monitor():
    try:
        while True:
            L_home.append(homes.count)
            yield env.timeout(1)
    except simpy.Interrupt:
        pass

### In order to start simulation with full or half full hospital, change `initial_patients` variable to `K//2` or `K`.

In [12]:
initial_patients = K

Initialize the system environment. 

In [13]:

env = simpy.Environment()
hospital = simpy.Resource(env, capacity=K)                # Hospital has K capacity.
homes = simpy.Resource(env, capacity=int(1e100))          # Homes has infinite capacity.
env.process(patient_generator(initial_patients))                          
L_monitor_process = env.process(L_monitor())
L_hosp_monitor_process = env.process(L_hosp_monitor())
L_home_monitor_process = env.process(L_home_monitor())


### Run the sytem for `run_time` units of time.

In [14]:
run_time = 1000
env.run(until=run_time) # run until run_time


### Run the sytem for `number_of_events`.

In [15]:
# number_of_events = 50
# while (len(Log) < number_of_events):
#     env.step()

Output of the system.
  - L: number of patients in the system at each second.
  - L_hosp: number of patients in the hospital at each second.
  - L_home: number of patients in their homes at each second.
  - Interarrival times: Interarrival times between consecutive patients.
  - Healing times: Indexes and healing times of patients.
  - Arrivals: Arrival times of patients.
  - Leaves: Indexes and leaving times of patients.
  - Log: Event list.

In [16]:
print(f"""
L: {L}

L_hosp: {L_hosp}

L_home: {L_home}

Interarrival times: {A[2:]}

Healing times: {S}

Arrivals: {A}

Leaves: {Leaves}
""")

print("Log: \n", "\n".join(map(str, Log)))
print (len(Log))
print (len(A))
print (len(S))



L: [0, 95, 84, 75, 67, 59, 56, 50, 51, 52, 49, 44, 48, 46, 49, 48, 45, 39, 37, 36, 39, 42, 45, 41, 41, 40, 46, 49, 47, 43, 41, 41, 38, 41, 47, 46, 47, 48, 50, 53, 56, 57, 56, 57, 48, 44, 40, 39, 40, 43, 44, 45, 47, 48, 45, 40, 36, 37, 33, 38, 35, 41, 42, 39, 49, 52, 53, 48, 48, 49, 47, 52, 45, 45, 47, 47, 46, 42, 42, 43, 44, 46, 44, 45, 48, 45, 48, 45, 47, 46, 45, 45, 46, 46, 44, 44, 47, 49, 49, 49, 50, 52, 50, 50, 49, 43, 41, 40, 37, 37, 39, 37, 37, 39, 40, 40, 42, 43, 43, 45, 48, 52, 51, 52, 53, 49, 45, 46, 44, 42, 41, 41, 41, 41, 41, 43, 42, 41, 42, 40, 45, 41, 44, 45, 43, 41, 38, 34, 33, 31, 33, 35, 32, 32, 34, 37, 37, 36, 37, 32, 30, 31, 33, 36, 36, 38, 37, 39, 33, 36, 36, 30, 30, 27, 26, 27, 31, 31, 30, 28, 33, 39, 39, 41, 42, 42, 45, 43, 47, 50, 45, 40, 37, 38, 38, 35, 33, 38, 39, 46, 45, 48, 49, 50, 47, 46, 45, 42, 48, 48, 47, 46, 44, 40, 43, 41, 39, 35, 37, 39, 41, 39, 34, 32, 35, 31, 29, 25, 22, 26, 22, 26, 28, 30, 31, 33, 31, 33, 40, 38, 35, 34, 39, 44, 43, 38, 37, 38, 35, 

Calculating model responses.

In [25]:
print(L_hosp.count(0) / len(L_hosp) * 100)   # long run probability of the hospital being empty,
print(sum(L_hosp) / len(L_hosp))             # average number of occupied beds in the hospital,
print(sum(L) / len(L))                       # average proportion of sick people in the population,
print(sum(S.values()) / len(S))              # total average sickness time 

0.33670033670033667
7.0
42.42424242424242
9.026175539673465
